In [1]:
# Imports
import pandas.io.sql as pds
import pandas as pd
import psycopg2 as pg2

In [2]:
# Initialize path to SQLite databasejdbc:sqlite:/C:/__tmp/test/sqlite/jdbcTest.db
#path = 'classic_rock.db'
#con = sq3.Connection(path)#

#con = pg2.connect("host=127.0.0.1 port=5432 dbname=shreeerp user=sa password=moraya")
cities_cnt = 0
countries_cnt = 0
temp_cnt = 0
cities = pd.read_csv("cities.csv",delimiter=",",header=0, dtype="string")
cities_cnt = cities.count
cities.info(verbose=True)
#with pd.ExcelWriter("test.xlsx",engine='xlsxwriter') as writer:
#    countries.to_excel(excel_writer=writer, sheet_name="new")
cities.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1245 entries, 0 to 1244
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   station_id  1245 non-null   string
 1   city_name   1244 non-null   string
 2   country     1245 non-null   string
 3   state       1217 non-null   string
 4   iso2        1239 non-null   string
 5   iso3        1245 non-null   string
 6   latitude    1245 non-null   string
 7   longitude   1245 non-null   string
dtypes: string(8)
memory usage: 77.9 KB


,station_id,city_name,country,state,iso2,iso3,latitude,longitude
0,41515,Asadabad,Afghanistan,Kunar,AF,AFG,34.8660000397,71.1500045859
1,38954,Fayzabad,Afghanistan,Badakhshan,AF,AFG,37.1297607616,70.5792471913
2,41560,Jalalabad,Afghanistan,Nangarhar,AF,AFG,34.4415269155,70.4361034738
3,38947,Kunduz,Afghanistan,Kunduz,AF,AFG,36.7279506623,68.8725296619
4,38987,Qala i Naw,Afghanistan,Badghis,AF,AFG,34.983000131,63.1332996367


In [3]:

countries = pd.read_csv("countries.csv",delimiter=",",header=0, dtype="string")
countries_cnt = countries.count
countries.head()



,country,native_name,iso2,iso3,population,area,capital,capital_lat,capital_lng,region,continent
0,Afghanistan,افغانستان,AF,AFG,26023100.0,652230.0,Kabul,34.526011,69.177684,Southern and Central Asia,Asia
1,Albania,Shqipëria,AL,ALB,2895947.0,28748.0,Tirana,41.326873,19.818791,Southern Europe,Europe
2,Algeria,الجزائر,DZ,DZA,38700000.0,2381741.0,Algiers,36.775361,3.060188,Northern Africa,Africa
3,American Samoa,American Samoa,AS,ASM,55519.0,199.0,Pago Pago,-14.275479,-170.70483,Polynesia,Oceania
4,Angola,Angola,AO,AGO,24383301.0,1246700.0,Luanda,-8.82727,13.243951,Central Africa,Africa


In [4]:
temp_facts = pd.read_parquet("daily_weather.parquet")
temp_facts.astype({'station_id': 'string',	'city_name': 'string',	'date': 'string',	'season': 'string',	'avg_temp_c': 'float64',	'min_temp_c': 'float64',	'max_temp_c': 'float64',	'precipitation_mm': 'float64',	'snow_depth_mm': 'float64',	'avg_wind_dir_deg': 'float64',	'avg_wind_speed_kmh': 'float64',	'peak_wind_gust_kmh': 'float64',	'avg_sea_level_pres_hpa': 'float64',	'sunshine_total_min': 'float64',})
temp_facts.head()

temp_cnt = temp_facts.count
print(temp_cnt)

#temp_small = temp_facts.iloc[:100]
#temp_small.to_csv("test.csv", sep=',', encoding='utf-8',)



<bound method DataFrame.count of       station_id city_name       date  season  avg_temp_c  min_temp_c  \
0          41515  Asadabad 1957-07-01  Summer        27.0        21.1   
1          41515  Asadabad 1957-07-02  Summer        22.8        18.9   
2          41515  Asadabad 1957-07-03  Summer        24.3        16.7   
3          41515  Asadabad 1957-07-04  Summer        26.6        16.1   
4          41515  Asadabad 1957-07-05  Summer        30.8        20.0   
...          ...       ...        ...     ...         ...         ...   
24216      67975  Masvingo 2023-09-01  Spring        19.5         9.6   
24217      67975  Masvingo 2023-09-02  Spring        21.3        10.5   
24218      67975  Masvingo 2023-09-03  Spring        22.1        13.0   
24219      67975  Masvingo 2023-09-04  Spring        21.5        13.1   
24220      67975  Masvingo 2023-09-05  Spring        20.0        14.0   

       max_temp_c  precipitation_mm  snow_depth_mm  avg_wind_dir_deg  \
0            35.6 

In [5]:
cities.set_index('country')
countries.set_index('country')
cities['country'].astype(str)
countries['country'].astype(str)
#city_country_df = cities.join(countries, on='country', how='inner', lsuffix='_city', rsuffix='_country')

city_country_df = pd.concat([cities, countries], axis=1, join='inner',)
city_country_df.head()

,station_id,city_name,country,state,iso2,iso3,latitude,longitude,country,native_name,iso2,iso3,population,area,capital,capital_lat,capital_lng,region,continent
0,41515,Asadabad,Afghanistan,Kunar,AF,AFG,34.8660000397,71.1500045859,Afghanistan,افغانستان,AF,AFG,26023100.0,652230.0,Kabul,34.526011,69.177684,Southern and Central Asia,Asia
1,38954,Fayzabad,Afghanistan,Badakhshan,AF,AFG,37.1297607616,70.5792471913,Albania,Shqipëria,AL,ALB,2895947.0,28748.0,Tirana,41.326873,19.818791,Southern Europe,Europe
2,41560,Jalalabad,Afghanistan,Nangarhar,AF,AFG,34.4415269155,70.4361034738,Algeria,الجزائر,DZ,DZA,38700000.0,2381741.0,Algiers,36.775361,3.060188,Northern Africa,Africa
3,38947,Kunduz,Afghanistan,Kunduz,AF,AFG,36.7279506623,68.8725296619,American Samoa,American Samoa,AS,ASM,55519.0,199.0,Pago Pago,-14.275479,-170.70483,Polynesia,Oceania
4,38987,Qala i Naw,Afghanistan,Badghis,AF,AFG,34.983000131,63.1332996367,Angola,Angola,AO,AGO,24383301.0,1246700.0,Luanda,-8.82727,13.243951,Central Africa,Africa


In [11]:
#pd.ExcelWriter("test.xlsx",sh)
#import numpy as np

#import pandas as pd
temp_facts2 = temp_facts.iloc[:10000]
temp_facts2.astype({'station_id': 'string',	'city_name': 'string',	'date': 'string',	'season': 'string',	'avg_temp_c': 'float64',	'min_temp_c': 'float64',	'max_temp_c': 'float64',	'precipitation_mm': 'float64',	'snow_depth_mm': 'float64',	'avg_wind_dir_deg': 'float64',	'avg_wind_speed_kmh': 'float64',	'peak_wind_gust_kmh': 'float64',	'avg_sea_level_pres_hpa': 'float64',	'sunshine_total_min': 'float64',})

#temp_facts2 = temp_facts.loc[lambda temp_facts: temp_facts['station_id'] == 41515  , :]
#temp_facts2.to_csv("test3.csv", sep=',', encoding='utf-8',)
#print(temp_facts2.head())
#print(cities.head())

#cities.to_csv("city-test.csv", sep=',', encoding='utf-8',)
#city_country_df.set_index('station_id')



#print(cities['station_id'].dtype)

#cities.info(verbose=True)
#cities_new = cities.convert_dtypes()
#cities['station_id'].apply(tostring())





#temp_facts2["station_id"].astype(str)
#temp_facts2['station_id'].map(str)

temp_facts2 = temp_facts2.astype({"station_id": 'string'})
#temp_facts2[['station_id']] = temp_facts2[['station_id']].astype(str)
cities = cities.astype({"station_id": 'string'})
#cities.info(verbose=True)

#temp_facts2.convert_dtypes(infer_objects=True, convert_string=True)
#temp_facts2.info(verbose=True)  

cities.set_index('station_id')
temp_facts.set_index('station_id')
#result_df = temp_facts.join(cities,  on='station_id', how='inner', lsuffix='_tt')
result_df = pd.merge(temp_facts2, cities, how='inner', on='station_id', suffixes=('_tmp',''))
print(result_df.head())

print(result_df.count())

#temp_small = result_df.iloc[:100]
result_df.to_csv("test5.csv", sep=',', encoding='utf-8',)

#join(countries, on='country_id', how='inner')
#res_cnt = result_df.count
#print(res_cnt)



  station_id city_name_tmp       date  season  avg_temp_c  min_temp_c  \
0      41515      Asadabad 1957-07-01  Summer        27.0        21.1   
1      41515      Asadabad 1957-07-02  Summer        22.8        18.9   
2      41515      Asadabad 1957-07-03  Summer        24.3        16.7   
3      41515      Asadabad 1957-07-04  Summer        26.6        16.1   
4      41515      Asadabad 1957-07-05  Summer        30.8        20.0   

   max_temp_c  precipitation_mm  snow_depth_mm  avg_wind_dir_deg  ...  \
0        35.6               0.0            NaN               NaN  ...   
1        32.2               0.0            NaN               NaN  ...   
2        35.6               1.0            NaN               NaN  ...   
3        37.8               4.1            NaN               NaN  ...   
4        41.7               0.0            NaN               NaN  ...   

   peak_wind_gust_kmh  avg_sea_level_pres_hpa  sunshine_total_min  city_name  \
0                 NaN                     

In [7]:
result_df.index.is_unique

True

In [8]:
import collections.abc
#hyper needs the four following aliases to be done manually.
collections.Iterable = collections.abc.Iterable
collections.Mapping = collections.abc.Mapping
collections.MutableSet = collections.abc.MutableSet
collections.MutableMapping = collections.abc.MutableMapping

import dfsql as ds

ds.sql_query("select count(station_id ) from a", a=result_df)



10000

In [9]:
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

q = "select count(*) from result_df"

cnt = pysqldf(q)

cnt.info()

#print(cnt)



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   count(*)  1 non-null      int64
dtypes: int64(1)
memory usage: 140.0 bytes


In [22]:
# Write the query
con = pg2.connect("host=127.0.0.1 port=5432 dbname=shreeerp user=sa password=moraya")
query = '''select * from core.docs limit 1000 '''

#cur = conn.cursor()
#cur.execute("select * from core.docs limit 2")
#a = cur.fetchall()

# Execute the query
docs = pds.read_sql(query, con)

docs.head()

docs.set_index('docid')

dept = pds.read_sql(''' select ceid as docdept, cename as dept_name from core.departments''', con)
dept.to_csv("dept.csv", sep=',', encoding='utf-8',)
dept.head()
dept.set_index('docdept')


### join is giving incorrect results after join
#join_df = docs.join(dept, how='inner', on='docdept', lsuffix='_l')
#join_df.to_csv("test6.csv", sep=',', encoding='utf-8',)
#join_df.head()

merge_df = pd.merge(docs, dept, how='inner', on='docdept')
merge_df.to_csv("test7.csv", sep=',', encoding='utf-8',)





C:\Users\aweso\AppData\Local\Temp\ipykernel_15268\3507169274.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  docs = pds.read_sql(query, con)
C:\Users\aweso\AppData\Local\Temp\ipykernel_15268\3507169274.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dept = pds.read_sql(''' select ceid as docdept, cename as dept_name from core.departments''', con)


,docid,siteid,docno,docsrno,docdept_l,doctype,transactiontype,docdate,docissuedto,coyear,...,pptype,fromdate,todate,totalearnings,totaldeductions,includeinbalcalc,stocktype,docstatus,docdept,dept_name
29,9390,0,None,NaN,3.0,28,0,2006-10-04,None,2,...,None,None,None,None,None,0,1,5,6,Agri
251,9449,0,None,NaN,4.0,97,0,2006-09-27,None,2,...,None,None,None,None,None,0,1,5,7,Civil
310,9450,0,None,NaN,4.0,97,0,2006-09-27,None,2,...,None,None,None,None,None,0,1,5,7,Civil
345,9391,0,None,NaN,3.0,28,0,2006-10-04,None,2,...,None,None,None,None,None,0,1,5,6,Agri
361,9452,0,None,NaN,1.0,28,0,2006-09-02,None,2,...,None,None,None,None,None,0,1,5,4,Account


In [ ]:
query='''
SELECT Artist, Release_Year, COUNT(*) AS num_songs, AVG(PlayCount) AS avg_plays  
    FROM rock_songs
    GROUP BY Artist, Release_Year
    ORDER BY num_songs desc;
'''

query = '''select * from core.docs limit 100'''

# Execute the query
observations_generator = pds.read_sql(query,
                            con,
                            coerce_float=True, # Doesn't efefct this dataset, because floats were correctly parsed
                            parse_dates=['docdate'], # Parse `Release_Year` as a date
                            chunksize=5 # Allows for streaming results as a series of shorter tables
                           )

for index, observations in enumerate(observations_generator):
    if index < 5:
        print(f'Observations index: {index}'.format(index))
        display(observations)